# Robustheit und Profi-Techniken

Du musst hier nicht alles sofort auswendig können.
Dieses Notebook ist eine **Werkzeugkiste**: Nimm dir zuerst 2-3 Werkzeuge, die dir direkt helfen.

## Lernweg im Notebook
- **Pflicht (für alle):** `try/except`, `else/finally`, einfache Validierung mit `raise`
- **Nice-to-have:** Logging, `with` (Kontextmanager)
- **Später nochmal anschauen:** Type Hints, `assert` und kleine Tests

## Lernziele
- Fehler sauber behandeln, statt Programme abstürzen zu lassen.
- Klare Fehlermeldungen und sinnvolle Validierung schreiben.
- Abläufe nachvollziehbar protokollieren (`logging`).
- Wiederkehrende Robustheitsmuster sicher anwenden.


## 0) Warum Robustheit wichtig ist

Auch guter Code trifft in der Praxis auf Probleme:
- falsche Eingaben
- fehlende Dateien
- ungültige Daten

Robuster Code bedeutet:
- Fehler klar erkennen
- freundlich reagieren
- sauber aufräumen


In [ ]:
# Kleines Beispiel: robust statt Absturz
werte = ["12", "abc", "0"]

for text in werte:
    try:
        zahl = int(text)
        print("100 /", zahl, "=", 100 / zahl)
    except ValueError:
        print(f"'{text}' ist keine ganze Zahl.")
    except ZeroDivisionError:
        print("Division durch 0 ist nicht erlaubt.")


### Mini-Checkpoint

- Frage: Welcher Fehler entsteht bei `int("abc")`?
- Frage: Welcher Fehler entsteht bei `100 / 0`?
- Mini-Aufgabe: Fange beide Fehler getrennt ab.


In [ ]:
# Deine Zelle



## 1) Pflicht: `try` und `except`

`try` ist für Code, der kaputtgehen kann.
`except` ist die Reaktion auf den Fehler.

Mini-Regel für Einsteiger:
**In `try` nur das, was wirklich kaputtgehen kann.**
Nicht 20 Zeilen auf einmal.


In [ ]:
# Kleines, sauberes Beispiel
text = "42"

try:
    zahl = int(text)
except ValueError:
    print("Keine gültige Zahl")
else:
    print("Konvertiert:", zahl)


### Schlecht vs. besser (Try-Block zu groß)

Schlecht:
- Ein großer `try` mit `except Exception` versteckt, was genau kaputt war.

Besser:
- Kleine `try`-Blöcke und konkrete Fehlertypen.


In [ ]:
def schlecht(text):
    try:
        zahl = int(text)
        ergebnis = 100 / zahl
        print("Ergebnis:", ergebnis)
    except Exception:
        print("Irgendein Fehler ist passiert.")


def besser(text):
    try:
        zahl = int(text)
    except ValueError as e:
        print("Ungültige Zahl:", e)
        return

    try:
        ergebnis = 100 / zahl
    except ZeroDivisionError as e:
        print("Division durch 0:", e)
        return

    print("Ergebnis:", ergebnis)


for t in ["10", "abc", "0"]:
    print("\nInput:", t)
    print("schlecht:")
    schlecht(t)
    print("besser:")
    besser(t)


### Warnung: Fehler nicht verstecken

Schlecht:
```python
try:
    ...
except:
    pass
```

Warum schlecht?
- Du siehst nicht, was kaputtging.
- Debugging wird schwer.

Besser:
- Konkreten Fehler fangen (`ValueError`, `FileNotFoundError`, ...)
- Mindestens Fehlermeldung ausgeben oder loggen.


In [ ]:
# Schlecht
try:
    int("12.5")
except:
    pass

# Besser
try:
    int("12.5")
except ValueError as e:
    print("Fehler sichtbar:", e)


### Du bist dran (`try/except`)

Aufgabe:
- Schreibe eine Funktion `parse_int(text)`.
- Bei gültiger Zahl: gib `int(text)` zurück.
- Bei Fehler: gib `None` zurück und drucke eine klare Meldung.


In [ ]:
# Deine Zelle



In [ ]:
# ✅ Lösung (optional)
def parse_int(text):
    try:
        return int(text)
    except ValueError as e:
        print(f"'{text}' ist keine ganze Zahl:", e)
        return None

print(parse_int("15"))
print(parse_int("abc"))


### Mini-Checkpoint (`try/except`)

- Frage: Warum ist `except ValueError` besser als `except Exception` für Zahl-Parsing?
- Mini-Aufgabe: Fange nur `ValueError` beim Umwandeln von `"3.14"` mit `int(...)`.


In [ ]:
try:
    int("3.14")
except ValueError as e:
    print("Nur ValueError abgefangen:", e)


## 2) Pflicht: `else` und `finally`

- `else` läuft nur, wenn **kein** Fehler passiert.
- `finally` läuft **immer**.

Merksatz:
`finally` ist dein "Aufräumen-Block".


In [ ]:
texte = ["25", "x"]

for text in texte:
    try:
        zahl = int(text)
    except ValueError:
        print(f"'{text}' ist ungültig.")
    else:
        print(f"'{text}' erfolgreich ->", zahl)
    finally:
        print("Aufräumen ... läuft immer")
        print("---")


In [ ]:
# Beispiel mit Datei: manuelles Schließen via finally
import os

os.makedirs("robustheit_demo", exist_ok=True)
pfad = os.path.join("robustheit_demo", "finally_demo.txt")

datei = None
try:
    datei = open(pfad, "w", encoding="utf-8")
    datei.write("Robuster Code mit finally\n")
    print("Datei geschrieben")
except OSError as e:
    print("Dateifehler:", e)
finally:
    if datei is not None:
        datei.close()
        print("Datei wurde geschlossen")


### Du bist dran (`else/finally`)

Aufgabe:
- Versuche, eine Zahl aus einem Text zu machen.
- Bei Erfolg: melde das im `else`.
- Gib in `finally` immer `"Aufräumen..."` aus.


In [ ]:
# Deine Zelle



In [ ]:
# ✅ Lösung (optional)
def konvertiere_text(text):
    try:
        zahl = int(text)
    except ValueError:
        print("Ungültig")
    else:
        print("Erfolg:", zahl)
    finally:
        print("Aufräumen...")

konvertiere_text("8")
konvertiere_text("x")


### Mini-Checkpoint (`else/finally`)

- Frage: Wann läuft `else`?
- Frage: Wann läuft `finally`?
- Mini-Aufgabe: Probiere den Code einmal mit gültiger und einmal mit ungültiger Eingabe.


In [ ]:
for text in ["11", "foo"]:
    try:
        zahl = int(text)
    except ValueError:
        print("Fehler bei", text)
    else:
        print("OK", zahl)
    finally:
        print("Immer")


## 3) Pflicht: Validierung mit `raise`

Wenn Eingaben ungültig sind, darfst du selbst einen Fehler auslösen:
`raise ValueError("...")`

Das macht Funktionen klarer und sicherer.


In [ ]:
def pruefe_alter_text(text):
    if text.strip() == "":
        raise ValueError("Eingabe darf nicht leer sein.")

    try:
        alter = int(text)
    except ValueError as e:
        raise ValueError("Alter muss eine ganze Zahl sein.") from e

    if alter <= 0:
        raise ValueError("Alter muss größer als 0 sein.")

    return alter

for t in ["21", "", "abc", "-5"]:
    try:
        print(t, "->", pruefe_alter_text(t))
    except ValueError as e:
        print(t, "-> Fehler:", e)


### Nice-to-have: Eigene Exception als besserer Name

Erst `ValueError` (Pflicht) verstehen.
Dann kannst du eigene Exception-Namen nutzen, wenn es fachlich klarer wird.


In [ ]:
class LeereEingabeError(Exception):
    pass


def validiere_name(name):
    if name.strip() == "":
        raise LeereEingabeError("Name darf nicht leer sein.")
    return name.title()

try:
    print(validiere_name("mia"))
    print(validiere_name("   "))
except LeereEingabeError as e:
    print("Eigene Exception abgefangen:", e)


### Du bist dran (`raise`)

Aufgabe:
- Schreibe `pruefe_menge(text)`.
- Fehlerfall 1: leerer Text.
- Fehlerfall 2: keine Zahl.
- Fehlerfall 3: Zahl ist negativ.
- Sonst: Zahl als `int` zurückgeben.


In [ ]:
# Deine Zelle



In [ ]:
# ✅ Lösung (optional)
def pruefe_menge(text):
    if text.strip() == "":
        raise ValueError("Menge darf nicht leer sein.")

    try:
        menge = int(text)
    except ValueError as e:
        raise ValueError("Menge muss eine ganze Zahl sein.") from e

    if menge < 0:
        raise ValueError("Menge darf nicht negativ sein.")

    return menge

for t in ["5", "", "abc", "-1"]:
    try:
        print(t, "->", pruefe_menge(t))
    except ValueError as e:
        print(t, "->", e)


### Mini-Checkpoint (`raise`)

- Frage: Warum ist `raise` besser als stilles Weiterlaufen bei falschen Daten?
- Mini-Aufgabe: Löse `ValueError("Preis fehlt")` aus, wenn ein Text leer ist.


In [ ]:
preis_text = ""

try:
    if preis_text.strip() == "":
        raise ValueError("Preis fehlt")
except ValueError as e:
    print("Abgefangen:", e)


## 4) Nice-to-have: Logging

Warum nicht nur `print()`?
- Keine Level (`INFO`, `WARNING`, `ERROR`)
- Schwer zu filtern
- Nicht automatisch in Datei

Mit Logging kannst du gezielt protokollieren.


In [ ]:
import logging
import os

os.makedirs("robustheit_demo", exist_ok=True)
log_pfad = os.path.join("robustheit_demo", "robustheit.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_pfad, mode="w", encoding="utf-8"),
        logging.StreamHandler(),
    ],
    force=True,
)

logger = logging.getLogger("robustheit")

logger.info("Programmstart")
logger.warning("Beispiel-Warnung")
logger.error("Beispiel-Fehler")

print("Logdatei:", log_pfad)


In [ ]:
# Kleines Beispiel: Warning bei falscher Eingabe

def parse_preis(text):
    try:
        return float(text)
    except ValueError:
        logger.warning("Ungültiger Preis: '%s'", text)
        return None

print(parse_preis("12.5"))
print(parse_preis("abc"))


In [ ]:
# Optional: DEBUG-Level (später)
logger.debug("Diese Nachricht siehst du erst ab Level DEBUG")


In [ ]:
# Logdatei anzeigen
with open(log_pfad, "r", encoding="utf-8") as f:
    print(f.read())


### Du bist dran (Logging)

Aufgabe:
- Schreibe eine Funktion `teile(a, b)`.
- Wenn `b == 0`: logge `WARNING` und gib `None` zurück.
- Sonst: gib das Ergebnis zurück.


In [ ]:
# Deine Zelle



In [ ]:
# ✅ Lösung (optional)
def teile(a, b):
    if b == 0:
        logger.warning("Division durch 0 verhindert: a=%s, b=%s", a, b)
        return None
    return a / b

print(teile(10, 2))
print(teile(10, 0))


### Mini-Checkpoint (Logging)

- Frage: Was ist der Vorteil von `WARNING` vs. `ERROR`?
- Mini-Aufgabe: Logge eine `INFO`-Meldung "Datenimport gestartet".


In [ ]:
logger.info("Datenimport gestartet")


### Bonus (optional): Debugging und Stack Trace

Wenn ein Fehler unklar ist, hilft ein Stack Trace:
- Welche Funktion hat welche aufgerufen?
- In welcher Zeile ist es passiert?


In [ ]:
import traceback


def schritt_1(x):
    return schritt_2(x)


def schritt_2(x):
    return schritt_3(x)


def schritt_3(x):
    return 100 / x


try:
    schritt_1(0)
except Exception:
    print("Stack Trace:")
    traceback.print_exc()


Mini-Aufgabe:
- Ändere den Aufruf auf `schritt_1(5)` und beobachte den Unterschied.


In [ ]:
print("Probiere: schritt_1(5)")
print("Ergebnis:", schritt_1(5))


## 5) Nice-to-have: Kontextmanager mit `with`

Merksatz:
`with` sorgt dafür, dass am Ende automatisch aufgeräumt wird (z. B. Datei schließen).


In [ ]:
import os

os.makedirs("robustheit_demo", exist_ok=True)
pfad = os.path.join("robustheit_demo", "zeilen.txt")

with open(pfad, "w", encoding="utf-8") as f:
    f.write("Erste Zeile\n")
    f.write("Zweite Zeile\n")
    f.write("Dritte Zeile\n")

with open(pfad, "r", encoding="utf-8") as f:
    zeilen = f.readlines()

print("Zeilenanzahl:", len(zeilen))


### Später nochmal anschauen: eigener Kontextmanager

Für den Einstieg reicht `with open(...)` völlig aus.
Eigene Kontextmanager kommen später.


In [ ]:
# Optionales Beispiel (später): eigener Kontextmanager
from contextlib import contextmanager

@contextmanager
def markierter_block(name):
    print(f"Start: {name}")
    try:
        yield
    finally:
        print(f"Ende: {name}")

with markierter_block("Demo"):
    print("Arbeit im Block")


### Du bist dran (`with`)

Aufgabe:
- Lies `zeilen.txt` mit `with`.
- Gib die Anzahl Zeilen aus.


In [ ]:
# Deine Zelle



In [ ]:
# ✅ Lösung (optional)
with open("robustheit_demo/zeilen.txt", "r", encoding="utf-8") as f:
    anzahl = len(f.readlines())

print("Anzahl Zeilen:", anzahl)
assert anzahl == 3


### Mini-Checkpoint (`with`)

- Frage: Was passiert automatisch am Ende von `with open(...)`?
- Mini-Aufgabe: Schreibe eine neue Datei `kurz.txt` mit genau einer Zeile.


In [ ]:
with open("robustheit_demo/kurz.txt", "w", encoding="utf-8") as f:
    f.write("Hallo\n")

print("Datei geschrieben")


## 6) Später nochmal anschauen: Type Hints

Type Hints sind ein **Hilfszettel** für Menschen und Tools.
Sie ändern nicht, wie Python grundsätzlich läuft.

Kurz gesagt:
- besser lesbar
- bessere Editor-Hinweise
- weniger Missverständnisse im Team


In [ ]:
def addiere(a: int, b: int) -> int:
    return a + b


def baue_index(namen: list[str]) -> dict[str, int]:
    return {name: i for i, name in enumerate(namen)}

print(addiere(2, 3))
print(baue_index(["Mia", "Noah", "Lea"]))


### Du bist dran (Type Hints)

Aufgabe:
- Ergänze Type Hints für diese Funktion:
```python
def kombiniere(vorname, nachname):
    return vorname + " " + nachname
```


In [ ]:
# Deine Zelle



In [ ]:
# ✅ Lösung (optional)
def kombiniere(vorname: str, nachname: str) -> str:
    return vorname + " " + nachname

print(kombiniere("Ada", "Lovelace"))


### Mini-Checkpoint (Type Hints)

- Frage: Warum funktionieren Programme auch ohne Type Hints?
- Mini-Aufgabe: Schreibe eine Funktion mit Signatur `def verdopple(werte: list[int]) -> list[int]: ...`


In [ ]:
def verdopple(werte: list[int]) -> list[int]:
    return [w * 2 for w in werte]

print(verdopple([1, 2, 3]))


## 7) Später nochmal anschauen: `assert` und Mini-Tests

`assert` ist für Tests und interne Annahmen.
Es ist **nicht** für Benutzereingaben gedacht.

Merksatz:
- Eingaben von außen: `try/except` + klare Meldung
- interne Logik testen: `assert`


In [ ]:
def quadratiere(n: int) -> int:
    return n * n

assert quadratiere(0) == 0
assert quadratiere(3) == 9
assert quadratiere(-4) == 16

print("Alle asserts erfolgreich")


Hinweis:
Für größere Test-Suiten nutzt man später meist `unittest` oder `pytest`.


### Du bist dran (`assert`)

Aufgabe:
- Schreibe eine Funktion `ist_gerade(n)`.
- Teste sie mit mindestens 3 `assert`-Zeilen.


In [ ]:
# Deine Zelle



In [ ]:
# ✅ Lösung (optional)
def ist_gerade(n: int) -> bool:
    return n % 2 == 0

assert ist_gerade(2) is True
assert ist_gerade(7) is False
assert ist_gerade(0) is True

print("assert-Tests für ist_gerade ok")


### Mini-Checkpoint (`assert`)

- Frage: Warum ist `assert` für Benutzereingaben ungeeignet?
- Mini-Aufgabe: Teste `addiere(2, 2) == 4` mit `assert`.


In [ ]:
assert addiere(2, 2) == 4
print("addiere-Test ok")


## 8) Praxisübungen (alltagsnah, mit Gerüst)

Jede Übung hat:
- Startcode
- kurze Tipps
- optionale Lösung


### Übung A: Eingabe-Parser für Alter (Pflicht)

Ziel:
- leer prüfen
- Zahl prüfen
- `> 0` prüfen

Starter-Gerüst:


In [ ]:
def lese_alter(text: str) -> int:
    # TODO: leer?
    # TODO: int umwandeln
    # TODO: > 0 prüfen
    return 0

# Beispielaufrufe
# print(lese_alter("21"))
# print(lese_alter(""))


Tipps:
- Bei Fehlern `raise ValueError("...")`.
- Möglichst klare Fehlermeldungen.


In [ ]:
# ✅ Lösung Übung A (optional)
def lese_alter(text: str) -> int:
    if text.strip() == "":
        raise ValueError("Alter fehlt.")

    try:
        alter = int(text)
    except ValueError as e:
        raise ValueError("Alter muss eine ganze Zahl sein.") from e

    if alter <= 0:
        raise ValueError("Alter muss größer als 0 sein.")

    return alter

for probe in ["21", "", "abc", "-5"]:
    try:
        print(probe, "->", lese_alter(probe))
    except ValueError as e:
        print(probe, "->", e)


### Übung B: Datei-Leser mit schöner Fehlermeldung (Pflicht)

Ziel:
- Datei lesen
- bei nicht vorhandener Datei klare Meldung ausgeben


In [ ]:
def lese_datei_sicher(pfad: str) -> str | None:
    # TODO: Datei lesen
    # TODO: FileNotFoundError behandeln
    return None

# Beispiel:
# print(lese_datei_sicher("robustheit_demo/zeilen.txt"))
# print(lese_datei_sicher("robustheit_demo/gibt_es_nicht.txt"))


Tipps:
- `except FileNotFoundError:`
- Gib einen Hinweis zurück, z. B. `None`.


In [ ]:
# ✅ Lösung Übung B (optional)
def lese_datei_sicher(pfad: str) -> str | None:
    try:
        with open(pfad, "r", encoding="utf-8") as f:
            return f.read()
    except FileNotFoundError:
        print(f"Datei nicht gefunden: {pfad}")
        return None

print(lese_datei_sicher("robustheit_demo/zeilen.txt"))
print(lese_datei_sicher("robustheit_demo/gibt_es_nicht.txt"))


### Übung C: JSON-Loader mit Logging (Nice-to-have)

Ziel:
- JSON-Text laden
- ungültiges JSON sauber melden
- Fehler als `ERROR` loggen


In [ ]:
import json


def lade_json_sicher(text: str):
    # TODO: json.loads(text)
    # TODO: JSONDecodeError behandeln
    # TODO: logger.error(...)
    return None

# Beispiel:
# print(lade_json_sicher('{"ok": true}'))
# print(lade_json_sicher('{'))


Tipps:
- Fehlerklasse: `json.JSONDecodeError`
- `logger.error("...")` reicht für den Anfang.


In [ ]:
# ✅ Lösung Übung C (optional)
import json


def lade_json_sicher(text: str):
    try:
        return json.loads(text)
    except json.JSONDecodeError as e:
        logger.error("Ungültiges JSON: %s", e)
        return None

print(lade_json_sicher('{"ok": true}'))
print(lade_json_sicher('{'))


### Übung D: Mini-Validator für E-Mail (ohne Regex, Pflicht)

Ziel:
- nicht leer
- muss `@` enthalten


In [ ]:
def validiere_email_simple(text: str) -> str:
    # TODO: leer?
    # TODO: @ enthalten?
    # TODO: sonst Text zurückgeben
    return text

# Beispiel:
# print(validiere_email_simple("max@example.com"))
# print(validiere_email_simple("maxexample.com"))


Tipps:
- Prüfe zuerst auf leeren Text.
- Dann `if "@" not in text:`


In [ ]:
# ✅ Lösung Übung D (optional)
def validiere_email_simple(text: str) -> str:
    if text.strip() == "":
        raise ValueError("E-Mail darf nicht leer sein.")
    if "@" not in text:
        raise ValueError("E-Mail muss ein '@' enthalten.")
    return text

for probe in ["max@example.com", "", "maxexample.com"]:
    try:
        print(probe, "->", validiere_email_simple(probe))
    except ValueError as e:
        print(probe, "->", e)


## 9) Typische Einsteigerfehler

1. `except:` + `pass`
- Fehler wird unsichtbar.

2. Zu großer `try`-Block
- Ursache schwer zu finden.

3. Alles mit `except Exception` fangen
- Zu breit für viele Fälle.

4. `assert` für Benutzereingaben nutzen
- Besser: `if` + `raise` + klare Meldung.

5. `os.listdir()` mit "nur Dateien" verwechseln
- Ordner sind auch dabei, ggf. filtern.

6. Logging vergessen
- Ohne Logs wird Support/Debugging deutlich schwerer.


## 10) Cheat Sheet (Spickzettel)

```python
# 1) try/except ValueError
try:
    zahl = int(text)
except ValueError as e:
    print("Ungültige Zahl:", e)

# 2) try/except/else/finally
try:
    zahl = int(text)
except ValueError:
    print("Fehler")
else:
    print("OK", zahl)
finally:
    print("Aufräumen")

# 3) raise
if preis < 0:
    raise ValueError("Preis darf nicht negativ sein")

# 4) logging
logger.info("Start")
logger.warning("Auffälligkeit")
logger.error("Fehler")

# 5) with
with open("datei.txt", "r", encoding="utf-8") as f:
    inhalt = f.read()

# 6) assert (für Tests)
assert quadratiere(3) == 9
```


## 11) Aufräumen (optional)

Wenn du Demo-Dateien löschen möchtest:
Achtung: Der Ordner `robustheit_demo` wird komplett entfernt.


In [ ]:
# Optional ausführen
# import os
# import shutil
#
# ordner = "robustheit_demo"
# if os.path.exists(ordner):
#     shutil.rmtree(ordner)
#     print("Ordner entfernt:", ordner)
# else:
#     print("Ordner nicht gefunden:", ordner)


## Zusammenfassung

- Starte mit den Pflicht-Werkzeugen: `try/except`, `else/finally`, `raise`.
- Nutze danach Logging und `with`, um Code besser wartbar zu machen.
- Type Hints und `assert` helfen dir zusätzlich bei Lesbarkeit und Tests.
- Arbeite immer in kleinen, klaren Schritten: prüfen, reagieren, aufräumen.

Wenn du diese Muster sicher beherrschst, schreibst du deutlich robusteren Python-Code.
